# PREPROCESSING FILE
- loads the datasets you choose,
- creates a dataframe
- resamples
- saves
#### Do this here, then use another notebook for individual tasks
### imports

In [1]:
import os
import pickle
import h5py
import time
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib import gridspec
import scipy.stats as stats
import math
import random
import gc
import multiprocessing as mp
import tensorflow as tf
%matplotlib inline

# load all data for DS08
- two files, dev and test set

In [2]:
h5_dir = 'data_h5/'
fnames = [
    'N-CMAPSS_DS01-005.h5',
    'N-CMAPSS_DS03-012.h5',
    'N-CMAPSS_DS04.h5',
    'N-CMAPSS_DS05.h5',
    'N-CMAPSS_DS06.h5',
    'N-CMAPSS_DS07.h5',
    'N-CMAPSS_DS08a-009.h5',
    'N-CMAPSS_DS08c-008.h5'
]

sets = ['dev', 'test']

df = pd.DataFrame()
ui = 0

for filename in fnames:
    print(filename)
    for _set in sets:
        print(_set)
        with h5py.File(h5_dir+filename, 'r') as hdf:
            a_data = np.array(hdf.get(f"A_{_set}"))
            w_data = np.array(hdf.get(f"W_{_set}"))
            x_data = np.array(hdf.get(f"X_s_{_set}"))
            xv_data = np.array(hdf.get(f"X_v_{_set}"))
            t_data = np.array(hdf.get(f"T_{_set}"))
            y_data = np.array(hdf.get(f"Y_{_set}"))

            a_labels = [l.decode('utf-8') for l in list(np.array(hdf.get('A_var')))]
            w_labels = [l.decode('utf-8') for l in list(np.array(hdf.get('W_var')))]
            x_labels = [l.decode('utf-8') for l in list(np.array(hdf.get('X_s_var')))]
            xv_labels = [l.decode('utf-8') for l in list(np.array(hdf.get('X_v_var')))]
            t_labels = [l.decode('utf-8') for l in list(np.array(hdf.get('T_var')))]
            
        df_a = DataFrame(data=a_data, columns=a_labels)
        df_a['ui'] = -1
        df_a['dataset'] = filename.split('_')[1].split('.')[0]
        df_w = DataFrame(data=w_data, columns=w_labels)
        df_x = DataFrame(data=x_data, columns=x_labels)
        df_xv = DataFrame(data=xv_data, columns=xv_labels)
        df_t = DataFrame(data=t_data, columns=t_labels)
        df_y = DataFrame(data=y_data, columns=['y'])
        print(f"<{filename}> : {pd.unique(df_a.unit)}")
        for n in list(pd.unique(df_a.unit)):
            df_a.loc[df_a['unit'] == n, 'ui'] = ui
            ui = ui + 1

        df_temp = pd.concat([df_a, df_y, df_w, df_x, df_xv, df_t], axis=1)
        #print(df_temp.head())
        if(len(df)) == 0:
            df = df_temp
        else:
            df = pd.concat([df, df_temp], axis=0)      
        
        del df_a, df_w, df_x, df_xv, df_t, df_y, a_data, w_data, t_data, x_data, y_data, df_temp

N-CMAPSS_DS01-005.h5
dev
<N-CMAPSS_DS01-005.h5> : [1. 2. 3. 4. 5. 6.]
test
<N-CMAPSS_DS01-005.h5> : [ 7.  8.  9. 10.]
N-CMAPSS_DS03-012.h5
dev
<N-CMAPSS_DS03-012.h5> : [1. 2. 3. 4. 5. 6. 7. 8. 9.]
test
<N-CMAPSS_DS03-012.h5> : [10. 11. 12. 13. 14. 15.]
N-CMAPSS_DS04.h5
dev
<N-CMAPSS_DS04.h5> : [1. 2. 3. 4. 5. 6.]
test
<N-CMAPSS_DS04.h5> : [ 7.  8.  9. 10.]
N-CMAPSS_DS05.h5
dev
<N-CMAPSS_DS05.h5> : [1. 2. 3. 4. 5. 6.]
test
<N-CMAPSS_DS05.h5> : [ 7.  8.  9. 10.]
N-CMAPSS_DS06.h5
dev
<N-CMAPSS_DS06.h5> : [1. 2. 3. 4. 5. 6.]
test
<N-CMAPSS_DS06.h5> : [ 7.  8.  9. 10.]
N-CMAPSS_DS07.h5
dev
<N-CMAPSS_DS07.h5> : [1. 2. 3. 4. 5. 6.]
test
<N-CMAPSS_DS07.h5> : [ 7.  8.  9. 10.]
N-CMAPSS_DS08a-009.h5
dev
<N-CMAPSS_DS08a-009.h5> : [1. 2. 3. 4. 5. 6. 7. 8. 9.]
test
<N-CMAPSS_DS08a-009.h5> : [10. 11. 12. 13. 14. 15.]
N-CMAPSS_DS08c-008.h5
dev
<N-CMAPSS_DS08c-008.h5> : [1. 2. 3. 4. 5. 6.]
test
<N-CMAPSS_DS08c-008.h5> : [ 7.  8.  9. 10.]


In [3]:
df.head()

,unit,cycle,Fc,hs,ui,dataset,y,alt,Mach,TRA,...,fan_eff_mod,fan_flow_mod,LPC_eff_mod,LPC_flow_mod,HPC_eff_mod,HPC_flow_mod,HPT_eff_mod,HPT_flow_mod,LPT_eff_mod,LPT_flow_mod
0,1.0,1.0,1.0,1.0,0,DS01-005,99,3013.0,0.376362,70.311996,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.000604,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,0,DS01-005,99,3020.0,0.376866,70.311996,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.000604,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,0,DS01-005,99,3025.0,0.377685,70.311996,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.000604,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,0,DS01-005,99,3035.0,0.376992,70.399887,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.000604,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,0,DS01-005,99,3043.0,0.377622,70.399887,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.000604,0.0,0.0,0.0


### get the labels

In [4]:
y_labels = t_labels
t_labels = []
t_labels.append(w_labels)
t_labels.append(x_labels)
t_labels = [l for labels in t_labels for l in labels]
print(y_labels)
print(t_labels)
print(xv_labels)

csv_dir = 'data_csv/'

with open(csv_dir + 'v_labels.txt', "w") as f:
    for l in xv_labels:
        f.write(f"{l}\n")
        
with open(csv_dir+'y_labels.txt', "w") as f:
    for l in y_labels:
        f.write(f"{l}\n")

with open(csv_dir+'t_labels.txt', "w") as f:
    for l in t_labels:
        f.write(f"{l}\n")

['fan_eff_mod', 'fan_flow_mod', 'LPC_eff_mod', 'LPC_flow_mod', 'HPC_eff_mod', 'HPC_flow_mod', 'HPT_eff_mod', 'HPT_flow_mod', 'LPT_eff_mod', 'LPT_flow_mod']
['alt', 'Mach', 'TRA', 'T2', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2', 'P21', 'P24', 'Ps30', 'P40', 'P50', 'Nf', 'Nc', 'Wf']
['T40', 'P30', 'P45', 'W21', 'W22', 'W25', 'W31', 'W32', 'W48', 'W50', 'SmFan', 'SmLPC', 'SmHPC', 'phi']


## Create the augmented auxiliary data by aggregating over units

In [5]:
df_max = df[['ui', 'Fc', 'unit', 'cycle', 'dataset']].groupby('ui').agg('max')
df_max.reset_index(inplace=True)
df_max.head()

,ui,Fc,unit,cycle,dataset
0,0,1.0,1.0,100.0,DS01-005
1,1,3.0,2.0,75.0,DS01-005
2,2,2.0,3.0,100.0,DS01-005
3,3,1.0,4.0,95.0,DS01-005
4,4,3.0,5.0,89.0,DS01-005


# DO ONCE, then proceed below
## LOAD the dataset and resample it, then SAVE it
- for some reason I could not stop getting duplicate index errors when trying to resample before saving and loading

In [ ]:
del df
df = pd.read_csv(csv_dir+'df08_all.csv')
df.drop(columns=[df.columns[0]], inplace=True)
df.index = pd.to_timedelta(df.index, unit='s')
df = df.resample('10S').interpolate(method='time')
df.head()